In [107]:
import pandas as pd
from itertools import permutations
from tqdm import tqdm
import numpy as np
import scipy.optimize as opt
import statsmodels.api as sm
import scipy.stats as stats
import PCIData
import matplotlib.pyplot as plt
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
import vectorbt as vbt
import pickle


class DataTable:
    def __init__(self, pairName:list, constituents:list):
        self.pairName = pairName
        self.constituents = constituents
        self.dict_data={}

    def getData(self):
        if self.dict_data == {}:
            self.setData()
        else:
            pass
        return self.dict_data
    
    def setData(self):
        self.dict_data = dict(zip(self.pairName, self.constituents))
        return self.dict_data

class Definition:
    def f_load_adjPrice(filepath) -> pd.DataFrame:
        """Load pandas dataframe of the price series
        Args:
          filepath (str): Filepath to the adjPrice_nasdaq100.csv"""
        try:
            adjPrice = pd.read_pickle(filepath)
            adjPrice.index = adjPrice["date"]
        except Exception as e:
            print("Error : {e}")
            adjPrice = None
        return adjPrice
    
    def f_selectTicker(tickerList, adjPrice)-> pd.DataFrame:
        """ 
        Select the columns that are in tickerList

        Args:
            tickerList (list): List of all tickers to take into considerations
            adjPrice (pd.DataFrame): Adjusted price of all tickers
        """
        
        return adjPrice.loc[:, adjPrice.columns.isin(tickerList)]

    def f_insample_outsample(data, year):
        data.index = pd.to_datetime(data.index)
        
        start_of_year = pd.to_datetime(str(year))
        
        end_of_period = start_of_year + pd.DateOffset(months=6)  # This is "year + 6 months"
        start_of_period = start_of_year - pd.DateOffset(years=4) 
        
        # Adjusted filter condition using the corrected datetime offsets
        all_sample = data[(data.index < end_of_period) & (data.index >= start_of_period)]
        
        # Cleaning operations
        tmp = all_sample.dropna(how="all")
        all_sample_cleaned = tmp.dropna(axis=1, how='any')
        
        
        # Select in-sample data: data from the four years prior to the specified year.
        # It checks if the year in the data is less than the specified year
        # and greater than or equal to four years before the specified year.
        
        in_sample = all_sample_cleaned[(all_sample_cleaned.index < start_of_year) & (all_sample_cleaned.index >= start_of_period)]
        
        # Select out-sample data: data from the specified year but only for the first six months.
        # It checks if the year in the data is exactly the specified year and
        # if the month is less than or equal to 6 (January to June).
        out_sample = all_sample_cleaned[(all_sample_cleaned.index >= start_of_year) & (all_sample_cleaned.index <= end_of_period)]

        # Return both the in-sample and out-sample datasets.
        return in_sample, out_sample


    def pairDatabase(consituentList : list) -> pd.DataFrame:
    
        combinations = [(a, b) for a, b in permutations(consituentList, 2)]

        # Create a DataFrame
        pairDatabase = pd.DataFrame(combinations, columns=["Stock 1", "Stock 2"])
        pairDatabase["pair"] = pairDatabase["Stock 1"] + "/" + pairDatabase["Stock 2"]
        
        return pairDatabase

class Computation:
    def __init__(self, X1, X2) -> None:
        self.X1 = X1
        self.X2 = X2

    def Rsquare(sigmaM, sigmaR, rho):
        return 2 * (sigmaM ** 2) / (2 * (sigmaM ** 2) + (1+rho) * sigmaR ** 2)
   
    def fit_mle(self, W, tol=0.001):
        '''
        Maximum likelihood estimation of the associated Kalman filter

        Parameters:
        W (numpy.ndarray): the time-series which we want to model has both permanent and transient components

        Returns:
        rho (float): estimated value of rho
        sigma_M (float): estimated value of sigma_M
        sigma_R (float): estimated value of sigma_R
        '''

        # Set empty list for estimate and ll values
        estimates = []
        lls = []

        # Set distributions for random guesses
        rnd_rho = stats.uniform(loc=-1, scale=2)
        std = np.std(np.diff(W)) # Compute standard deviation of "pci process" (or residuals)
        rnd_sigma = stats.norm(loc=std, scale=std / 2) # Why scaled to std / 2? Can we find better? ####################

        # Define function for generating random initial guesses from above distributions
        def gen_x_i():
            return rnd_rho.rvs(), rnd_sigma.rvs(), rnd_sigma.rvs()

        # Minimize the negative log-likelihood function
        bounds = ((-1, 1), (0, np.inf), (0, np.inf)) # Set bounds for opt.
        x_i = self.lagvar_estimate_par(W) # Get initial guesses using lagged variance equations
        res = opt.minimize(self.f_to_min_par , x0=(x_i), args=(W), bounds=bounds, tol=tol) # Minimize -ll function

        if res.success: # If optimization is a success
            estimates.append(res.x) # Save estimates in list
            lls.append(-res.fun)  # Save log-likelihood in list
        # return res.x
    
        # Repeat optimization with different random initial values
    
        n_att = 0
        while len(lls) < 10 and n_att < 100:
            n_att += 1
            x_i = gen_x_i()
            res = opt.minimize(self.f_to_min_par, x0=(x_i), args=(W), bounds=bounds, tol=tol)

            if res.success:  # If optimization is a success
                estimates.append(res.x)  # Save estimates
                lls.append(-res.fun)  # Save log-likelihood

        try:
            argmax = np.argmax(lls)  # Find index of max likelihood
            return estimates[argmax] # Return estimates linked to max likelihood
        except:
            # print('Estimation failed!')
            return len(x0) * [np.nan]  # Returns nanns
        

    def lagvar_estimate_par(self, W):
        '''
        Estimate parameters of partial AR model using lagged variances. Used for first estimation of parameters

        Parameters
        W (numpy.ndarray): A partial autoregressive time series

        Returns:
        rho_lv (float): estimated value for rho
        sigma_M_lv (float): estimated value for sigma_M
        sigma_R_lv (float): estimated value for sigma_R
        '''

        # See Matthew Clegg: "Modeling Time Series with both Permanent and Transient
        # Component using the Partially Autoregressive Model". See equations on page 5.

        # Calculate variance of the lagged differences. Left hand side of equation (3)
        v1 = np.var(W[1:] - W[:-1])
        v2 = np.var(W[2:] - W[:-2])
        v3 = np.var(W[3:] - W[:-3])

        # Calculate rho from v1, v2, v3. Equations (4), page 5
        rho_lv = -(v1 - 2 * v2 + v3) / (2 * v1 - v2)

        # Calculate sigma_M. Equations (4), page 5
        if (rho_lv + 1) / (rho_lv - 1) * (v2 - 2 * v1) > 0:
            sigma_M_lv = np.sqrt(1 / 2 * (rho_lv + 1) / (rho_lv - 1) * (v2 - 2 * v1))
        else:
            sigma_M_lv = 0

        # Calculate sigma_M. Equations (4), page 5
        if v2 > 2 * sigma_M_lv ** 2:
            sigma_R_lv = np.sqrt(1 / 2 * (v2 - 2 * sigma_M_lv ** 2))
        else:
            sigma_R_lv = 0

        return rho_lv, sigma_M_lv, sigma_R_lv

    def fit_ols_on_diff(self):
        '''
        Fits an OLS model on the first differences of time series X1 and X2

        Parameters:
        X1 (numpy.ndarray): price time-series
        X2 (numpy.ndarray): price time-series

        Returns:
        results.params[0]: returns the Beta value of our OLS fit
        '''
        ret_X1 = np.diff(self.X1)
        ret_X2 = np.diff(self.X2)

        results = sm.OLS(ret_X2, ret_X1).fit()

        return results.params[0]

    def kalman_estimate(self, W, rho, sigma_M, sigma_R):
        '''
        Calculate estimates of mean-reverting series (M_t) and random walk series (R_t).

        Parameters:
        X (numpy.ndarray): A partial autoregressive time-series
        rho (float): AR(1) coefficient / mean reversion coefficient.
        sigma_M (float): standard deviation of the innovations of the mean-reverting component
        sigma_R (float): standard deviation of the innovations of the random walk component

        Returns:
        M (numpy.ndarray): An estimate of the mean reverting component of our time series
        R (numpy.ndarray): An estimate of the random walk component of our time series
        eps (numpy.ndarray): Prediction errors for each time step
        '''

        # See Matthew Clegg: "Modeling Time Series with both Permanent and Transient
        # Component using the Partially Autoregressive Model". See algo on page 9.

        # Create arrays for storing both components and prediction errors
        M = np.zeros(len(W))
        R = np.zeros(len(W))
        eps = np.zeros(len(W))

        # Set state at t=0
        if sigma_R == 0: # If series is purely mean-reverting
            M[0] = W[0]
            R[0] = 0
        else:
            M[0] = 0
            R[0] = W[0]

        # Calculate Kalman gains
        if sigma_M == 0: # If series is purely a random walk
            K_M = 0
            K_R = 1
        elif sigma_R == 0: # If series is purely mean-reverting
            K_M = 1
            K_R = 0
        else:
            # See equations (11), page 8
            sqr = np.sqrt((1 + rho) ** 2 * sigma_R ** 2 + 4 * sigma_M ** 2)
            K_M = 2 * sigma_M ** 2 / (sigma_R * (sqr + rho * sigma_R + sigma_R) + 2 * sigma_M ** 2) # Compute gain for M
            K_R = 2 * sigma_R / (sqr - rho * sigma_R + sigma_R) # Compute gain for R

        # Calculate estimates recursively
        for i in range(1, len(W)):
            w_hat = rho * M[i - 1] + R[i - 1] # Predicted value of W[i]
            eps[i] = W[i] - w_hat # Prediction error
            M[i] = rho * M[i - 1] + eps[i] * K_M
            R[i] = R[i - 1] + eps[i] * K_R

        return M, R, eps

    def calc_log_like(self, W, rho, sigma_M, sigma_R):
        '''
        Compute negative log likelihood function

        Parameters:
        X (numpy.ndarray): A partial autoregressive time series
        rho (float): AR(1) coefficient / mean reversion coefficient.
        sigma_M (float): standard deviation of the innovations of the mean-reverting component
        sigma_R (float): standard deviation of the innovations of the random walk component

        Returns:
        ll (float): Value of the log likelihood, a measure of goodness of fit for our model
        '''

        N = len(W)
        _, _, eps = self.kalman_estimate(W, rho, sigma_M, sigma_R) # Compute the prediction errors for each time step

        # Compute the value of the log-likelihood function
        ll = -(N - 1) / 2 * np.log(2 * np.pi * (sigma_M ** 2 + sigma_R ** 2)) - 1 / (
                2 * (sigma_M ** 2 + sigma_R ** 2)) * np.sum(eps[1:] ** 2)

        return ll

    def f_to_min_par(self, x_i, W):
        rho, sigma_M, sigma_R = x_i
        '''
        Define the function to minimize for PAR model
        '''
        return -self.calc_log_like(W, rho, sigma_M, sigma_R)

    def f_to_min_pci(self, x_i, X1, X2):
        '''
        Define function to minimize
        '''
        if len(x_i) == 5 :
            alpha, beta, rho, sigma_M, sigma_R = x_i
            
        elif len(x_i) == 3:
            alpha, beta, sigma_R = x_i
            rho = 0
            sigma_M = 0
        
        W = X2 - beta * X1 - alpha
        return -self.calc_log_like(W, rho, sigma_M, sigma_R)

    def fit_pci(self, tol=0.001, LRT_mode: bool = True):
        '''
        Fit partial cointegrated model to time series X1 and X2 such that:
            - X_2,t = alpha + beta * X_1,t + W_t
            - W_t = M_t + R_t
            - M_t = rho * M_t-1 + eps(M_t)
            - R_t = R_t-1 + eps(R_t)
            - eps(M_t) ∼ NID(0, sigma_M)
            - eps(R_t) ∼ NID(0, sigma_R)

        Parameters:
        X1 (numpy.ndarray): time series
        X2 (numpy.ndarray): time series, supposedly partially cointegrated with X1

        Returns:
        alpha (float): estimated value for alpha (linear regression parameter)
        beta (float): estimated value for beta (linear regression parameter)
        rho (float): estimated AR(1) coefficient / mean reversion coefficient.
        sigma_M (float): standard deviation of the innovations of the mean-reverting component
        sigma_R (float): standard deviation of the innovations of the random walk component
        '''

        # Calculate initial guess for beta with linear regression
        results = self.fit_ols_on_diff()
        beta_i = results

        # Calculate initial guess for alpha
        alpha_i = self.X2[0] - beta_i * self.X1[0]

        # Calculate residuals W and initial guesses for rho, sigma_M, sigma_R (params_i)
        W = self.X2 - alpha_i - beta_i * self.X1
        params_i = self.fit_mle(W)

        rho, sigma_M, sigma_R = params_i

        # perform optimization depending on the mode (Complete Model)
        x_i = (alpha_i, beta_i, rho, sigma_M, sigma_R)  # initial guess
        bounds = [(None, None), (None, None), (-1, 1), (0.0001, None), (0.0001, None)]
        res = opt.minimize(self.f_to_min_pci, x_i, args=(self.X1, self.X2), tol=tol, bounds=bounds)
        alpha, beta, rho, sigma_M, sigma_R = res.x
        ll_model = -res.fun

        # W = self.X2 - alpha - beta * self.X1

        if LRT_mode:
            # perform optimization for the random Walk H0)
            # perform optimization depending on the mode (Complete Model)
            x_i = (alpha_i, beta_i, sigma_R)  # initial guess
            bounds = [(None, None), (None, None), (0.0001, None)]
            res = opt.minimize(self.f_to_min_pci, x_i, args=(self.X1, self.X2), tol=tol, bounds=bounds)
            ll_randomWalk = -res.fun
            lrt = ll_randomWalk - ll_model
            return alpha, beta, rho, sigma_M, sigma_R, ll_model, ll_randomWalk, lrt

        return alpha, beta, rho, sigma_M, sigma_R, ll_model
    
    def f_get_elligibility(Stock1, Stock2, inSample):
        
        X = inSample[[Stock1, Stock2]].dropna()

        X1 = X[Stock1]
        X2 = X[Stock2]

        coint = Computation(X1, X2)
        alpha_hat, beta_hat, rho_hat, sigma_M_hat, sigma_R_hat, ll_model, ll_randomWalk, ll_ratio = coint.fit_pci(LRT_mode = True)
        # M_hat, R_hat, _ = coint.kalman_estimate(W_hat, rho_hat, sigma_M_hat, sigma_R_hat)

        Rsquare_res = Computation.Rsquare(sigma_M_hat, sigma_R_hat, rho_hat)

        # print([ll_randomWalk, ll_model])
        
        # for pair in pairName:

        return {"elligibility":[rho_hat, Rsquare_res, ll_ratio],
            "estimators" :[alpha_hat, beta_hat, rho_hat, sigma_M_hat, sigma_R_hat]}
    
    def f_compute_estimate(pairName: pd.DataFrame, inSample: pd.DataFrame, pairData: pd.DataFrame) -> tuple :
        N = len(pairName)
        elligibilityData = np.zeros((N, 3))
        estimationData = np.zeros((N, 5))
        
        

        for pair in tqdm(range(N), desc="Processing Pairs"):
            try:
                pairName_i = pairName[pair]
                # print(f"Doing : {pairName_i}")
                Stock1 = pairData["Stock 1"][pair]
                Stock2 = pairData["Stock 2"][pair]
                
                dict_res = Computation.f_get_elligibility(Stock1, Stock2, inSample)  # Call f_get_elligibility using Computation class
                elligibilityData[pair] = dict_res["elligibility"]
                # latentVariablesData[pairName[pair]] = dict_res["latentVariables"]
                estimationData[pair] = dict_res["estimators"]
                #print(f"{pairName_i}: Done")
            except Exception as e: 
                print(f"Error with {pairName_i} and {e}")
        return elligibilityData, estimationData


    def f_create_dataframe(elligibilityData: pd.DataFrame, estimationData: pd.DataFrame, pairName: pd.DataFrame)-> tuple:
    
        elligibilityData = pd.DataFrame(elligibilityData, index = pairName, columns=["rho", "Rsquare", "ll_ratio"])
        elligibilityData["ll_ratio"].fillna(0)
        elligibility_sorted_data = elligibilityData[
                                                    (elligibilityData["rho"] > 0.5) & 
                                                     (elligibilityData["Rsquare"] > 0.5)
                                                ].sort_values(by="ll_ratio", ascending=True)
        # Dataframe des paramètres estimés
        estimatorData = pd.DataFrame(estimationData, index = pairName, columns=["alpha_hat", "beta_hat", "rho_hat", "sigma_M_hat", "sigma_R_hat"])
        
        return elligibility_sorted_data, estimatorData
    
    def f_pairSelected(elligibility_sorted_data, n_keep :int):
        # Nombre de pair elligible
        n_pair = len(elligibility_sorted_data)
        if n_pair == 0:
                print("Elligibility_sorted_data is empty")
                return None
        
        n_elligible = len(elligibility_sorted_data.index)
        
        n = max(1, min(n_pair, n_keep))
        
        pairListSelected = elligibility_sorted_data.index[:n].to_list()

        return pairListSelected
    
    def get_stock_list(pairData, col):
        selected_pairs = pairData.loc[pairData["pair"].isin(col)]
        tmp = selected_pairs[["Stock 1", "Stock 2"]].values.flatten().tolist()
        stock_list = list(set(tmp))
        return stock_list
    
    def get_stock1_stock2(pair, pairData):
        S1, S2 = pairData.loc[pairData["pair"] == pair, ["Stock 1", "Stock 2"]].iloc[0].values
        return S1, S2
    
    def get_stock_price(df_price, stockList):
        return df_price[stockList].iloc[:, 0], df_price[stockList].iloc[:, 1]
    
    
    def f_create_Mt(pairListSelected, estimatorData, pairData, stock_price):
        
        estimator_Data_selected = estimatorData.loc[estimatorData.index.isin(pairListSelected)]
        
        n = len(pairListSelected)
        df_mt = np.zeros((len(stock_price), n))
        df_hedge = np.zeros((len(stock_price), n))

        for i in range(n):
            pair_tmp = pairListSelected[i]
            
            S1, S2 = Computation.get_stock1_stock2(pair_tmp, pairData)

            X1, X2 = Computation.get_stock_price(stock_price, [S1, S2])

            # Retrieve estimater
            alpha, beta, rho, sigmaM, sigmaR = estimator_Data_selected.loc[pair_tmp].to_list()

            W = X2 - alpha - beta * X1
            coint_tmp = Computation(X1, X2)
            _, Mt, _ = coint_tmp.kalman_estimate(W, rho, sigmaM, sigmaR)
            df_mt[:, i] = Mt
            # Number of share invested in S1 for 1$ invested in X2
            df_hedge[:, i] = beta / X2 # The amount of share
            
        return pd.DataFrame(data = df_mt, columns=pairListSelected), pd.DataFrame(data = df_hedge, columns = pairListSelected)
      
    
    def rolling_z_score(series: pd.Series, window: int) -> pd.Series:
        """
        Calculate the rolling Z-score of a given time series.
        
        Parameters:
        series (pd.Series): The time series data.
        window (int): The size of the rolling window.
        
        Returns:
        pd.Series: A series of rolling Z-scores.
        """
        # Convert numpy array to pandas Series if necessary
        # Convert numpy array to pandas Series if necessary
    # Initialize an array to hold the rolling Z-scores
        z_scores = np.full(series.shape, np.nan)

        # Loop through the series to calculate the rolling mean and standard deviation
        for i in range(window - 1, len(series)):
            window_values = series[i - window + 1 : i+1]
            #print([window_values[-1], series[i]])
            window_mean = np.mean(window_values)
            window_std = np.std(window_values, ddof=1)

            # Avoid division by zero
            if window_std != 0:
                z_scores[i] = (series.iloc[i] - window_mean) / window_std

        return z_scores
    
    def percentile_zscore(value, data):
        # Calculate the percentile-based z-score for a single value
        
        percentile = np.sum(data < value) / (len(data) - 1)  # Calculate percentile
        zscore = percentile * 2 - 1   # Transform to [-1, 1] range
        return zscore
    
    
    def calculate_zscores_historic(historical_data, initial_in_sample_size = 2):
        # Traiter nan values
        z = historical_data.copy()
        mask = historical_data.isna()
        non_nan_values_sample = historical_data[~mask]
        
        # Ensure there's enough historical data for the initial in-sample calculation
        if len(non_nan_values_sample) < initial_in_sample_size :
            raise ValueError("Not enough historical data for the specified initial in-sample size.")
        
        # Calculate in-sample z-scores for the initial baseline
        
        sample = list(non_nan_values_sample[:initial_in_sample_size])
        out_sample_data = list(non_nan_values_sample[initial_in_sample_size:])

        zscores = [Computation.percentile_zscore(x, np.array(sample)) for x in sample]
        
        # Calculate out-of-sample z-scores for any additional data
        for value in out_sample_data:
            # Update in-sample data with each new data point for out-of-sample calculation
            sample.append(value)
            # Calculate z-score for the new data point
            zscores.append(Computation.percentile_zscore(value, np.array(sample)))
        
        z[~mask] = zscores
        
        return z
    
    
    
class vbt_strategyPCI:
        
    def f_pre_vectorBT(pairListSelected, estimatorData, pairData, inSample ,outSample) -> dict:
        
        Mt_inSample, hedgeS1_inSample = Computation.f_create_Mt(pairListSelected, estimatorData, pairData, inSample)
        Mt_outSample, hedgeS1_outSample = Computation.f_create_Mt(pairListSelected, estimatorData, pairData, outSample)
        
        stockList = Computation.get_stock_list(pairData, pairListSelected)
        
        dict_output = { "Mt":       {"inSample" : Mt_inSample,
                                    "outSample" : Mt_outSample},
                        "hedgeS1": {"inSample" : hedgeS1_inSample,
                                    "outSample": hedgeS1_outSample},
                        "Price": {"inSample": inSample[stockList],
                                "outSample": outSample[stockList]}}
        
        return dict_output


    def custom_indicator(Mt : np.array, window: int = 20):
        """
        La fonction qui sert comme indicateur sur mesure pour vectorBT 
        """
        
        Mt_df = pd.DataFrame(Mt)
        
        # Mt_zscore = Mt_df.apply(func=lambda x: Computation.calculate_zscores_historic(x, initial_in_sample_size=2), axis=0) # Normalisé entre -1 et 1
        Mt_zscore = Mt_df.apply(func=lambda x: Computation.rolling_z_score(x, window), axis=0) # Normalisé 
        return Mt_zscore


    def f_get_Mt_normalize(Mt : dict, window: int) -> vbt.indicators.factory:

        strategyBase = vbt.IndicatorFactory(
                                                class_name = "StrategyBase",
                                                short_name = "StratBase",
                                                input_names = ["Mt"],
                                                param_names= ["window"],
                                                output_names = ["z_score"],
                                                ).from_apply_func(
                                                                vbt_strategyPCI.custom_indicator,
                                                                window = window
                                                                )
        strategy = strategyBase.run(Mt)
        return strategy



    def f_get_signals(strategy: vbt.indicators.factory , entry_threshold: float, exit_threshold: float) -> tuple:
        
        # Generate entry signals (Z-score crosses above the entry threshold)
        upper_crossed = strategy.z_score.vbt.crossed_above(entry_threshold)

        # Generate exit signals (Z-score crosses below the exit threshold)
        lower_crossed = strategy.z_score.vbt.crossed_below(exit_threshold)

        clean_upper_crossed, clean_lower_crossed = upper_crossed.vbt.signals.clean(lower_crossed)  
        
        return (clean_upper_crossed, clean_lower_crossed)


    def f_backtest(dict_pre_backtest : dict, type_back : str, hedgeS1: pd.DataFrame, price: pd.DataFrame, pairBacktested: str, pairData: pd.DataFrame) -> vbt.portfolio.base.Portfolio:

            
            clean_upper_crossed = dict_pre_backtest[type_back]["clean_upper_crossed"]
            clean_lower_crossed = dict_pre_backtest[type_back]["clean_lower_crossed"]
            
            positions_fill = price.copy()
            positions_fill.iloc[:, :] = False

            # Now, for sizing, long_entries, etc., we need independent copies of positions_fill
            sizing = positions_fill.copy()
            sizing.iloc[:, :] = 0
            long_entries = positions_fill.copy()
            short_entries = positions_fill.copy()
            short_exits = positions_fill.copy()
            long_exits = positions_fill.copy()

            # Directly access the first column name (pair)

            S1, S2 = Computation.get_stock1_stock2(pairBacktested, pairData)

            # Les prix
            X1, X2 = Computation.get_stock_price(price, [S1, S2])

            s1_sizing =  hedgeS1.loc[:, pairBacktested]

            s2_sizing = 1 / X2

            sizing.loc[clean_upper_crossed[pairBacktested].values, S2] =  s2_sizing.values[clean_upper_crossed[pairBacktested]]

            sizing.loc[clean_upper_crossed[pairBacktested].values, S1] = s1_sizing.values[clean_upper_crossed[pairBacktested].values]

            short_entries.loc[clean_upper_crossed[pairBacktested].values, S2] = True
            long_entries.loc[clean_upper_crossed[pairBacktested].values, S1] = True

            short_exits.loc[clean_lower_crossed[pairBacktested].values, S2] = True
            long_exits.loc[clean_lower_crossed[pairBacktested].values, S1] = True


            pf = vbt.Portfolio.from_signals(
                price,
                entries = long_entries,
                exits = long_exits,
                short_entries = short_entries,
                short_exits= short_exits,
                size = sizing,
                size_type= "amount",
                fees= 0.001,
                slippage=0.001)  # This line integrates your sizing logic

            return {"pf": pf, "Price": {"X1" : X1, "X2": X2}, "positions" : {"entries" : short_entries.loc[:, S2], "exits": short_exits.loc[:, S2]}, "sizing": sizing} 
        
        

    def f_pre_backtest(dict_output : dict, window : int, entry_threshold: float, exit_threshold: float) -> dict:
            
        # 2 Normaliser nos time series Mt pour chaque pair à l'aide vbt indicator factory
        
        Mt_inSample = dict_output["Mt"]["inSample"]
        strategy_inSample = vbt_strategyPCI.f_get_Mt_normalize(Mt = Mt_inSample, window = window)
        
        Mt_outSample = dict_output["Mt"]["outSample"]
        strategy_outSample = vbt_strategyPCI.f_get_Mt_normalize(Mt = Mt_outSample, window = window)
        
        
        clean_upper_crossed_inSample, clean_lower_crossed_inSample = vbt_strategyPCI.f_get_signals(strategy= strategy_inSample , entry_threshold = entry_threshold, exit_threshold = exit_threshold)
        
        clean_upper_crossed_outSample, clean_lower_crossed_outSample = vbt_strategyPCI.f_get_signals(strategy= strategy_outSample , entry_threshold = entry_threshold, exit_threshold = exit_threshold)

        return {"inSample": {"clean_upper_crossed" : clean_upper_crossed_inSample, "clean_lower_crossed" : clean_lower_crossed_inSample, "Mt": strategy_inSample},
                "outSample" : {"clean_upper_crossed" : clean_upper_crossed_outSample, "clean_lower_crossed" : clean_lower_crossed_outSample, "Mt": strategy_outSample}}

In [108]:
#def main(path: str, year: int) -> None:
# Load adjPrice for all tickers
year  = 2008
path = "/Users/sebastiencaron/Desktop/PCI-Project/strategyV1/data/"
filepath1 = f"{path}adjPrice_nasdaq100.pkl"
adjPrice = Definition.f_load_adjPrice(filepath1)


# Load the CSV file : Compustat daily consituent
filepath2 = f"{path}NasdaqConstituent.csv"
object_nasdaq = PCIData.DataCollection(filepath2)
# Liste des consituents pour une certaine année

consituent2008 = object_nasdaq.list_nasdaq_constituents_by_year(year)

# Get price series for list of tickers
df_2008 = Definition.f_selectTicker(consituent2008, adjPrice)

# Split In and Out sample
df_2008_in_sample, df_2008_out_sample = Definition.f_insample_outsample(df_2008, year)

# Get the In and Out sample 
df_2008_inout = pd.concat([df_2008_in_sample, df_2008_out_sample])

# Save in stock prices dictionnary
stockPrices = {"inSample": df_2008_in_sample,
            "outSample": df_2008_out_sample}

# Get the list of consituent of price that we have
consituentList = df_2008_in_sample.columns.tolist()

# Generate all possible permutations of pairs (including reversed pairs)
pairData = Definition.pairDatabase(consituentList=consituentList)

pairName = pairData["pair"]
inSample = stockPrices["inSample"]
pairTest  = pairName[0:1000]

# Takes a lot of time to run
# elligibilityData, estimationData = Computation.f_compute_estimate(pairTest, inSample, pairData)
with open('data.pickle', 'rb') as f:
    elligibilityData, estimationData = pickle.load(f)



### To add in an other category
elligibility_sorted_data, estimatorData =  Computation.f_create_dataframe(elligibilityData, estimationData, pairTest)

# Number of pair we keep
n_keep = 10

pairListSelected = Computation.f_pairSelected(elligibility_sorted_data, n_keep=n_keep)

stockList = Computation.get_stock_list(pairData, pairListSelected)


### VECTOR BT PART

#1) Sortir les informations : Mt, hedgeS1, price of inSample et outSample
dict_data = vbt_strategyPCI.f_pre_vectorBT(pairListSelected, estimatorData, pairData, stockPrices["inSample"], stockPrices["outSample"])

""" #2) Normaliser nos time series Mt pour chaque pair à l'aide vbt indicator factory
window = 30
Mt_inSample = dict_data["Mt"]["inSample"]
strategy = f_get_Mt_normalize(Mt = Mt_inSample, window = window)

#3) Get signals : entry and exit 
clean_upper_crossed, clean_lower_crossed = f_get_signals(strategy= strategy , entry_threshold = 0.1, exit_threshold = -0.1)
"""

window = 30
entry_threshold = 0.1
exit_threshold = -0.1
dict_pre_backtest = vbt_strategyPCI.f_pre_backtest(dict_output = dict_data, window  = window, entry_threshold = entry_threshold, exit_threshold = exit_threshold)


#4) Get portfolio backtest
type_back = "inSample"
hedgeS1_inSample = dict_data["hedgeS1"][type_back]
price = dict_data["Price"][type_back]
pairBacktested = pairListSelected[0]
pf_dict = vbt_strategyPCI.f_backtest(dict_pre_backtest = dict_pre_backtest,type_back = type_back, hedgeS1 = hedgeS1_inSample, price = price, pairBacktested = pairBacktested, pairData=pairData)
pf = pf_dict["pf"]

In [109]:
pf.stats()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)
/var/folders/hv/1_92kz110w572ncggsb_c81c0000gp/T/ipykernel_758/3705677322.py:1: UserWarning: Object has multiple columns. Aggregating us

Start                         2004-01-02 00:00:00
End                           2007-12-31 00:00:00
Period                                       1006
Start Value                                 100.0
End Value                              100.010916
Total Return [%]                         0.010916
Benchmark Return [%]                   240.959293
Max Gross Exposure [%]                   0.040807
Total Fees Paid                          0.008645
Max Drawdown [%]                         0.378481
Max Drawdown Duration                       508.0
Total Trades                             7.090909
Total Closed Trades                      6.909091
Total Open Trades                        0.181818
Open Trade PnL                           -0.00224
Win Rate [%]                            60.526316
Best Trade [%]                          37.643311
Worst Trade [%]                        -20.212666
Avg Winning Trade [%]                    7.720681
Avg Losing Trade [%]                    -5.051475


### Code to add

In [14]:
# path = "/Users/sebastiencaron/Desktop/PCI-Project/strategyV1/data/"
# Mt, price_tmp = main(path= path, year= 2008)
price_tmp = stockPrices["inSample"][stockList]

## Vector BT pro

In [80]:

import vectorbt as vbt 



In [81]:
#1) Sortir les informations : Mt, hedgeS1, price of inSample et outSample
dict_data = f_pre_vectorBT(pairListSelected, estimatorData, pairData, stockPrices["inSample"], stockPrices["outSample"])

""" #2) Normaliser nos time series Mt pour chaque pair à l'aide vbt indicator factory
window = 30
Mt_inSample = dict_data["Mt"]["inSample"]
strategy = f_get_Mt_normalize(Mt = Mt_inSample, window = window)

#3) Get signals : entry and exit 
clean_upper_crossed, clean_lower_crossed = f_get_signals(strategy= strategy , entry_threshold = 0.1, exit_threshold = -0.1)
"""

window = 30
entry_threshold = 0.1
exit_threshold = -0.1
dict_pre_backtest = f_pre_backtest(dict_output = dict_data, window  = window, entry_threshold = entry_threshold, exit_threshold = exit_threshold)


#4) Get portfolio backtest
type_back = "inSample"
hedgeS1_inSample = dict_data["hedgeS1"][type_back]
price = dict_data["Price"][type_back]
pairBacktested = pairListSelected[0]
pf= f_backtest(dict_pre_backtest = dict_pre_backtest,type_back = type_back, hedgeS1 = hedgeS1_inSample, price = price, pairBacktested = pairBacktested)

In [79]:
pf.stats()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)
/var/folders/hv/1_92kz110w572ncggsb_c81c0000gp/T/ipykernel_758/3705677322.py:1: UserWarning: Object has multiple columns. Aggregating us

Start                         2004-01-02 00:00:00
End                           2007-12-31 00:00:00
Period                                       1006
Start Value                                 100.0
End Value                              100.011643
Total Return [%]                         0.011643
Benchmark Return [%]                    12.066851
Max Gross Exposure [%]                   0.021623
Total Fees Paid                          0.012882
Max Drawdown [%]                          0.09114
Max Drawdown Duration                       364.5
Total Trades                                 11.5
Total Closed Trades                          11.5
Total Open Trades                             0.0
Open Trade PnL                                0.0
Win Rate [%]                            63.043478
Best Trade [%]                           9.194193
Worst Trade [%]                        -12.388993
Avg Winning Trade [%]                    2.414652
Avg Losing Trade [%]                    -3.107956


In [75]:
# Si on est capable d'aggréé le log de trade, on va être capable d'avoir la performance de la stratégie pour plusieurs titres.

print(pf.orders.records_readable)

     Order Id Column  Timestamp      Size     Price      Fees  Side
0           0    ADP 2004-02-27  0.023557  42.40755  0.000999  Sell
1           1    ADP 2004-03-11  0.023557  42.08204  0.000991   Buy
2           2    ADP 2004-04-02  0.022599  44.20575  0.000999  Sell
3           3    ADP 2004-05-19  0.022599  44.50446  0.001006   Buy
4           4    ADP 2004-08-04  0.024231  41.22873  0.000999  Sell
..        ...    ...        ...       ...       ...       ...   ...
179       179   AABA 2007-10-24  0.003771  30.64932  0.000116  Sell
180       180   AABA 2007-10-30  0.003648  30.86083  0.000113   Buy
181       181   AABA 2007-11-09  0.003648  25.76421  0.000094  Sell
182       182   AABA 2007-11-14  0.003742  25.09507  0.000094   Buy
183       183   AABA 2007-11-15  0.003742  25.39458  0.000095  Sell

[184 rows x 7 columns]


## Créer un streamlite pour sélectionner la pair et voir le backtest

## Archives

In [185]:
positions_fill = price_tmp.copy()
positions_fill.iloc[:,:] = False

sizing = positions_fill

long_entries = positions_fill
short_entries = positions_fill
short_exits = positions_fill
long_exits = positions_fill

short_entries.loc[clean_upper_crossed[pair].values, S2] = True
long_entries.loc[clean_upper_crossed[pair].values, S1] = True

short_exits.loc[clean_lower_crossed[pair].values, S2] = True
long_exits.loc[clean_lower_crossed[pair].values, S1] = True

col = [t[:] for t in clean_upper_crossed.columns]

# Commencer avec le vecteur de position et après décider si le stock va long ou short, car sinon il peut avoir des mixed signals. 
# Trouver l'array des hedge ratios pour chq pair

for pair in col:

     # Les tickers
     S1, S2 = Computation.get_stock1_stock2(pair, pairData)
     
     # Les prix
     X1, X2 = Computation.get_stock_price(price_tmp, [S1, S2])
     
     s1_sizing = hedgeS1.loc[pair]
     
     s2_sizing = 1 / X2
     
     sizing.loc[clean_upper_crossed[pair].values, S2] = - 1 * s2_sizing[clean_upper_crossed[pair]]
     
     sizing.loc[clean_upper_crossed[pair].values, S1] = s1_sizing.loc[clean_upper_crossed[pair].values]
     
     short_entries.loc[clean_upper_crossed[pair].values, S2] = True
     long_entries.loc[clean_upper_crossed[pair].values, S1] = True

     short_exits.loc[clean_lower_crossed[pair].values, S2] = True
     long_exits.loc[clean_lower_crossed[pair].values, S1] = True
     
     


     
"""
exits = long_exits,
short_entries = short_entries,
short_exits= short_exits
"""

pf = vbt.Portfolio.from_signals(
     price_tmp,
     entries = long_entries,
     exits = long_exits,
     short_entries = short_entries,
     short_exits= short_exits
)